In [146]:
import pandas as pd 
import numpy as np
from datetime import datetime
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from publicsuffix2 import get_public_suffix
from tld import is_tld

In [147]:
import pandas 
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

In [148]:
def check_org (x):
    if x.count('.') == 1:
        return(x.split('.')[0])
    if x.count('.') == 2:
        if is_tld('.'.join(x.split('.')[1:])):return(x.split('.')[0])
        else:return(x.split('.')[1])
    else: return(x)
def check_tld (x):
    if x.count('.') == 1:
        return(x.split('.')[1])
    if x.count('.') == 2:
        if is_tld('.'.join(x.split('.')[1:])):return('.'.join(x.split('.')[1:]))
        else:return(x.split('.')[2])
    else: return(x)
def check_add_info (x):
    if x.count('.') == 2:
        if is_tld('.'.join(x.split('.')[1:])):return('0')
        else:return(x.split('.')[0])
    else: return('0')

In [149]:
# Feature correction 

train_df = pd.read_csv('train.csv', )
test_df = pd.read_csv('test.csv', )
# train_df.drop('Unnamed: 0',axis=1,inplace = True)
# test_df.drop('Unnamed: 0',axis=1,inplace = True)
test_df['label'] = 'test_df'
df = train_df.append(test_df, ignore_index=True)
# df[df ['Unnamed: 0'] == 40]
df['chars_in_subject'] = df['chars_in_subject'].fillna(10)
df.org = df.org.fillna('org_unknown')
df['Ratio-SubtoBody'] = df['chars_in_body'] / (df['chars_in_subject'] + df['chars_in_body'])
df['tld'] = df['tld'].fillna('notfound.com').apply(get_public_suffix)
df['date_tz'] = df['date'].apply(lambda x: x.split('(')[1][:3] if '(' in x else 'not_found')
import datefinder
# df['day'] = df['date'].apply(lambda x:x.split(',')[0])
def find_date(x):
    for x in datefinder.find_dates(x):
        return(x)
        break

df['date'] = df['date'].apply(find_date)
df['date'] = pd.to_datetime(df['date'], utc = True)
df['year'] = pd.DatetimeIndex(df['date']).year
df['qtofyr'] = pd.DatetimeIndex(df['date']).quarter
df['weekofyr'] = pd.DatetimeIndex(df['date']).weekofyear
df['month'] = pd.DatetimeIndex(df['date']).month
df['weekday'] = pd.DatetimeIndex(df['date']).dayofweek
df['hour'] = pd.DatetimeIndex(df['date']).hour
df['monthday'] = pd.DatetimeIndex(df['date']).dayofyear
df['second'] = pd.DatetimeIndex(df['date']).hour
df.drop('date',axis=1,inplace=True)
df['working_weekday'] = df['weekday'].apply(lambda x: 1 if x<6 else 0)
df['full_wsite'] = df['org'] +'.'+ df['tld'] 
df['org'] = df['full_wsite'].apply(check_org)
df['tld_2'] = df['full_wsite'].apply(check_tld)
df['add_info'] = df['full_wsite'].apply(check_add_info)

C:\Users\arind\AppData\Local\Temp\ipykernel_44112\1191904190.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = train_df.append(test_df, ignore_index=True)
C:\Users\arind\anaconda3\lib\site-packages\publicsuffix2\__init__.py:376: UserWarning: This function returns the private suffix, SLD, or registrable domain. This equivalent to function get_sld(). To get the public suffix itself, use get_tld().
  warnings.warn(
C:\Users\arind\AppData\Local\Temp\ipykernel_44112\1191904190.py:26: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series. To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  df['weekofyr'] = pd.DatetimeIndex(df['date']).weekofyear


In [150]:
df['work_hour'] = df['hour'].apply(lambda x:1 if (x>9) & (x<17) else 0)
df = pd.get_dummies(df, prefix=['MailType'], columns = ['mail_type'],drop_first=False)
df['tld_front'] = df.tld.apply(lambda x: x.split('.')[0] if '.' in x else '0')
df['i/u_ratio'] = (df['images']+1)/(df['urls'] + 1)
right_df = pd.read_csv('final_label_probab_df.csv')
df = df.merge(right_df, on='org', how='left', indicator=False)
a = df.dtypes.to_frame('type').reset_index()
a[(a['type']=='object') ]['index'].to_list()

['org',
 'tld',
 'label',
 'date_tz',
 'full_wsite',
 'tld_2',
 'add_info',
 'tld_front']

In [151]:
df.isna().sum()

Unnamed: 0                        0
org                               0
tld                               0
ccs                               0
bcced                             0
images                            0
urls                              0
salutations                       0
designation                       0
chars_in_subject                  0
chars_in_body                     0
label                             0
Ratio-SubtoBody                   0
date_tz                           0
year                              0
qtofyr                            0
weekofyr                          0
month                             0
weekday                           0
hour                              0
monthday                          0
second                            0
working_weekday                   0
full_wsite                        0
tld_2                             0
add_info                          0
work_hour                         0
MailType_Multipart/Alternati

In [152]:
df

,Unnamed: 0,org,tld,ccs,bcced,images,urls,salutations,designation,chars_in_subject,...,tld_front,i/u_ratio,label_0_probab,label_1_probab,label_2_probab,label_3_probab,label_4_probab,label_5_probab,label_6_probab,label_7_probab
0,0,ebay,ebay.in,0,0,35,120,0,0,49.0,...,ebay,0.297521,0.059908,0.504608,0.430876,0.000000,0.004608,0.000000,0.0,0.00000
1,1,efinmail,efinmail.com,0,0,1,7,0,0,107.0,...,efinmail,0.250000,0.492537,0.455224,0.052239,0.000000,0.000000,0.000000,0.0,0.00000
2,2,usebackpack,com,0,0,4,17,0,0,35.0,...,0,0.277778,0.448915,0.504142,0.046943,0.000000,0.000000,0.000000,0.0,0.00000
3,3,granular,ai,0,0,0,0,0,0,15.0,...,0,1.000000,0.026502,0.733216,0.001767,0.238516,0.000000,0.000000,0.0,0.00000
4,4,github,com,1,0,2,11,0,0,49.0,...,0,0.250000,0.190280,0.182867,0.000000,0.626853,0.000000,0.000000,0.0,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114536,34360,quora,com,0,0,8,110,1,1,134.0,...,0,0.081081,0.089127,0.408306,0.047597,0.000000,0.000000,0.000000,0.0,0.45497
114537,34361,sdconnect,sdconnect.in,0,0,23,117,1,0,52.0,...,sdconnect,0.203390,0.000000,0.500000,0.500000,0.000000,0.000000,0.000000,0.0,0.00000
114538,34362,iiitd,ac.in,0,0,1,4,1,0,35.0,...,ac,0.400000,0.006318,0.556185,0.000552,0.436093,0.000401,0.000451,0.0,0.00000
114539,34363,tatadocomo,com,0,0,0,6,1,0,38.0,...,0,0.142857,0.666667,0.166667,0.166667,0.000000,0.000000,0.000000,0.0,0.00000


In [132]:
df.drop(['org', 'tld'],axis = 1)

,Unnamed: 0,ccs,bcced,images,urls,salutations,designation,chars_in_subject,chars_in_body,label,...,tld_front,i/u_ratio,label_0_probab,label_1_probab,label_2_probab,label_3_probab,label_4_probab,label_5_probab,label_6_probab,label_7_probab
0,0,0,0,35,120,0,0,49.0,80027,2,...,ebay,0.297521,0.059908,0.504608,0.430876,0.000000,0.004608,0.000000,0.0,0.00000
1,1,0,0,1,7,0,0,107.0,2961,1,...,efinmail,0.250000,0.492537,0.455224,0.052239,0.000000,0.000000,0.000000,0.0,0.00000
2,2,0,0,4,17,0,0,35.0,25149,1,...,0,0.277778,0.448915,0.504142,0.046943,0.000000,0.000000,0.000000,0.0,0.00000
3,3,0,0,0,0,0,0,15.0,635296,1,...,0,1.000000,0.026502,0.733216,0.001767,0.238516,0.000000,0.000000,0.0,0.00000
4,4,1,0,2,11,0,0,49.0,2355,1,...,0,0.250000,0.190280,0.182867,0.000000,0.626853,0.000000,0.000000,0.0,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114536,34360,0,0,8,110,1,1,134.0,89070,test_df,...,0,0.081081,0.089127,0.408306,0.047597,0.000000,0.000000,0.000000,0.0,0.45497
114537,34361,0,0,23,117,1,0,52.0,24241,test_df,...,sdconnect,0.203390,0.000000,0.500000,0.500000,0.000000,0.000000,0.000000,0.0,0.00000
114538,34362,0,0,1,4,1,0,35.0,4558,test_df,...,ac,0.400000,0.006318,0.556185,0.000552,0.436093,0.000401,0.000451,0.0,0.00000
114539,34363,0,0,0,6,1,0,38.0,2591,test_df,...,0,0.142857,0.666667,0.166667,0.166667,0.000000,0.000000,0.000000,0.0,0.00000


In [133]:
# df['full_wsite'] = df['full_wsite'].apply(lambda x: '.'.join(x.split('_')) if '.' not in x.split('_')[1] else x)

In [134]:
df

,Unnamed: 0,org,tld,ccs,bcced,images,urls,salutations,designation,chars_in_subject,...,tld_front,i/u_ratio,label_0_probab,label_1_probab,label_2_probab,label_3_probab,label_4_probab,label_5_probab,label_6_probab,label_7_probab
0,0,ebay,ebay.in,0,0,35,120,0,0,49.0,...,ebay,0.297521,0.059908,0.504608,0.430876,0.000000,0.004608,0.000000,0.0,0.00000
1,1,efinmail,efinmail.com,0,0,1,7,0,0,107.0,...,efinmail,0.250000,0.492537,0.455224,0.052239,0.000000,0.000000,0.000000,0.0,0.00000
2,2,usebackpack,com,0,0,4,17,0,0,35.0,...,0,0.277778,0.448915,0.504142,0.046943,0.000000,0.000000,0.000000,0.0,0.00000
3,3,granular,ai,0,0,0,0,0,0,15.0,...,0,1.000000,0.026502,0.733216,0.001767,0.238516,0.000000,0.000000,0.0,0.00000
4,4,github,com,1,0,2,11,0,0,49.0,...,0,0.250000,0.190280,0.182867,0.000000,0.626853,0.000000,0.000000,0.0,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114536,34360,quora,com,0,0,8,110,1,1,134.0,...,0,0.081081,0.089127,0.408306,0.047597,0.000000,0.000000,0.000000,0.0,0.45497
114537,34361,sdconnect,sdconnect.in,0,0,23,117,1,0,52.0,...,sdconnect,0.203390,0.000000,0.500000,0.500000,0.000000,0.000000,0.000000,0.0,0.00000
114538,34362,iiitd,ac.in,0,0,1,4,1,0,35.0,...,ac,0.400000,0.006318,0.556185,0.000552,0.436093,0.000401,0.000451,0.0,0.00000
114539,34363,tatadocomo,com,0,0,0,6,1,0,38.0,...,0,0.142857,0.666667,0.166667,0.166667,0.000000,0.000000,0.000000,0.0,0.00000


In [135]:
a[(a['type']=='object') ]

,index,type
1,org,object
2,tld,object
11,label,object
13,date_tz,object
23,full_wsite,object
24,tld_2,object
25,add_info,object
41,tld_front,object


In [136]:
for x in a[(a['type']=='object') ]['index'].to_list(): 
    if x != 'label':
        le_org = preprocessing.LabelEncoder()
        df[x] = le_org.fit_transform(df[x])
df = df.set_index('Unnamed: 0', drop=True)
to_scale = []
for col in df.columns:
    if len(df[col].unique())>2:
        if 'probab' not in col:
            to_scale.append(col)
 

In [162]:
df.groupby(['add_info', 'label']).count().sort_values('add_info',ascending=False)['org'].to_csv('add_info_check.csv')

In [138]:
to_scale_stsc = [
 'images',
 'urls',
 'chars_in_subject',
 'chars_in_body','Ratio-SubtoBody']

In [139]:
to_scale_nosc = list(set(to_scale) - set(to_scale_stsc))

In [140]:
from sklearn.preprocessing import StandardScaler
for val in to_scale_stsc:
    sc = StandardScaler()
#     df[val] = sc.fit_transform(df[val])
    df[val] = sc.fit_transform(df[[val]])
# sc.transform

In [141]:
for val in to_scale_nosc:
    
#     df[val] = sc.fit_transform(df[val])
    if val!= 'label':
        df[val] = (df[val] - df[val].mean())/df[val].max()

In [142]:
to_scale_nosc

['month',
 'date_tz',
 'weekofyr',
 'tld_front',
 'add_info',
 'org',
 'year',
 'weekday',
 'qtofyr',
 'tld_2',
 'monthday',
 'i/u_ratio',
 'tld',
 'hour',
 'second',
 'ccs',
 'full_wsite',
 'label']

In [143]:
df = df.fillna(1/7)
train_df = df[df['label'] != 'test_df']
train_df['label'] = train_df['label'].astype('int')
test_df = df[df['label'] == 'test_df']

C:\Users\arind\AppData\Local\Temp\ipykernel_44112\2176746312.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['label'] = train_df['label'].astype('int')


In [144]:
train_df.to_csv('To_run_train.csv')
test_df.to_csv('To_run_test.csv')

In [125]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostClassifier
from sklearn.linear_model import Lasso


In [145]:
df.columns

Index(['org', 'tld', 'ccs', 'bcced', 'images', 'urls', 'salutations',
       'designation', 'chars_in_subject', 'chars_in_body', 'label',
       'Ratio-SubtoBody', 'date_tz', 'year', 'qtofyr', 'weekofyr', 'month',
       'weekday', 'hour', 'monthday', 'second', 'working_weekday',
       'full_wsite', 'tld_2', 'add_info', 'work_hour',
       'MailType_Multipart/Alternative', 'MailType_Multipart/Mixed',
       'MailType_Text/Html', 'MailType_multipart/IDM',
       'MailType_multipart/alternative', 'MailType_multipart/mixed',
       'MailType_multipart/related', 'MailType_multipart/report',
       'MailType_multipart/signed', 'MailType_text/HTML',
       'MailType_text/calendar', 'MailType_text/html', 'MailType_text/html ',
       'MailType_text/plain', 'tld_front', 'i/u_ratio', 'label_0_probab',
       'label_1_probab', 'label_2_probab', 'label_3_probab', 'label_4_probab',
       'label_5_probab', 'label_6_probab', 'label_7_probab'],
      dtype='object')